In [1]:
import os
import numpy as np
env_var = os.environ
os.environ["LACE_REPO"] = "/nfs/pic.es/user/l/lcabayol/DESI/LaCE"
os.environ["LACE_MANAGER_REPO"] = "/nfs/pic.es/user/l/lcabayol/DESI/LaCE_manager"

In [2]:
from lace.emulator.nn_architecture import MDNemulator_polyfit
from lace.emulator.nn_emulator import NNEmulator
from lace.emulator.gp_emulator import GPEmulator

from lace.emulator.emulator import P1D_emulator

from lace.emulator.test_simulation_nn import test_sim
from lace.emulator import pnd_archive

/data/astro/scratch/lcabayol/anaconda3/envs/DESIenv6/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# our modules
from lace.cosmo import fit_linP
from lace.emulator import poly_p1d
from lace.archive import pnd_archive
from lace.archive import interface_archive

In [4]:
emuparams = ['Delta2_p', 'n_p','mF', 'sigT_Mpc', 'gamma', 'kF_Mpc']

# CREATE TRAINING AND TESTING ARCHIVE

In [5]:
archive = pnd_archive.archivePND(sim_suite="Cabayol23")
archive.get_training_data()
len(archive.training_data)

9900

In [6]:
archive_test = pnd_archive.archivePND(sim_suite="Cabayol23", pick_sim='central')
archive_test.get_testing_data()
len(archive_test.testing_data)

11

## TRAIN NEURAL NETWORK EMULATOR  

By default, the NNemulator is trained with the following parameters:ç
- zmax=4.5
- Nsim=30 
- nepochs=100
- step_size=75
- kmax_Mpc=4
- ndeg=5. For the extended version (kmax_Mpc=8), set to 7
- postprocessing='768'. To train with the 1D post-processing change to '500'

- initial_weights=True. Always starts from the same iniial parameters. These were selected randomly. Set to False if random initialization is required.

In [7]:
emulator = NNEmulator(archive,emuparams,ndeg=5, save_path=None, nepochs=1)


start the training of the emulator
Training network on 9900
Emualtor trained in 2.09765625 seconds


## LOAD A NEURAL NETWORK EMULATOR

In [8]:
emulator = NNEmulator(archive,emuparams,ndeg=5, save_path=None, nepochs=1, model_path='NNmodels/NNEmulator_LaCEHC.pt', train=False)


Model loaded. No training needed


## TEST NEURAL NETWORK EMULATOR

In [9]:
p1d, p1derr=emulator.emulate_p1d_Mpc(archive_test.testing_data[0])

# Gaussian Process passing an archive

- It is trained on 330 P1D. By default, it does not use optical-depth rescalings
- The default options for $k$, $z$, and 'deg' are the same as for the NN emulator

In [10]:
archive = pnd_archive.archivePND(sim_suite="Pedersen21")
archive.get_training_data()
len(archive.training_data)

330

In [11]:
emulator = GPEmulator(passarchive=archive)

 /data/astro/scratch/lcabayol/anaconda3/envs/DESIenv6/lib/python3.10/site-packages/paramz/parameterized.py:61: RuntimeWarning:Don't forget to initialize by self.initialize_parameter()!


Training GP on 330 points
GPs optimised in 0.50 seconds


In [12]:
kMpc = archive_test.testing_data[0]['k_Mpc'][1:43]

In [13]:
p1d = emulator.emulate_p1d_Mpc(archive_test.testing_data[0],kMpc)

In [14]:
p1d

array([2.50225822, 2.22316626, 1.95884804, 1.7639055 , 1.61445488,
       1.49443691, 1.39441155, 1.30866012, 1.23355067, 1.16667041,
       1.10635151, 1.05140053, 1.00093728, 0.95429518, 0.91095743,
       0.87051494, 0.83263777, 0.79705536, 0.76354251, 0.7319092 ,
       0.70199321, 0.6736545 , 0.64677099, 0.62123536, 0.59695249,
       0.57383749, 0.55181412, 0.53081352, 0.51077316, 0.49163604,
       0.47334995, 0.45586689, 0.43914262, 0.4231362 , 0.40780965,
       0.39312768, 0.37905739, 0.36556806, 0.35263094, 0.34021912,
       0.32830731, 0.31687176])

# Gaussian Process without passing an archive

In [15]:
emulator = GPEmulator()

Training GP on 330 points
GPs optimised in 0.47 seconds


In [16]:
p1d = emulator.emulate_p1d_Mpc(archive_test.testing_data[0],kMpc)

In [17]:
p1d

array([2.53661373, 2.2236054 , 1.95150444, 1.75447542, 1.60464945,
       1.48485746, 1.38527753, 1.30004194, 1.22545645, 1.15908176,
       1.09923936, 1.04473237, 0.9946806 , 0.94841896, 0.90543283,
       0.86531542, 0.82773912, 0.79243555, 0.75918148, 0.72778875,
       0.69809673, 0.66996686, 0.64327837, 0.61792513, 0.59381305,
       0.5708582 , 0.54898518, 0.5281259 , 0.50821852, 0.48920665,
       0.47103865, 0.45366704, 0.43704803, 0.42114111, 0.40590871,
       0.39131586, 0.37733001, 0.36392072, 0.35105954, 0.33871979,
       0.32687642, 0.3155059 ])

# MODULE COMBIING NN AND GP EMULATORS

In [23]:
def P1D_emulator(archive=None, emu_algorithm=None, archive_label = 'Gadget', emulator_label=None, zmax=4.5, kmax_Mpc=4, ndeg=5, train=True, emu_type='polyfit', model_path=None, save_path=None, nepochs_nn=100):
    
    
    
    if emulator_label=='Pedersen21':
        print('Select emulator used in Pedersen et al. 2021')
        emuparams = ['Delta2_p', 'n_p', 'mF', 'sigT_Mpc', 'gamma', 'kF_Mpc']
        archive = pnd_archive.archivePND(sim_suite="Pedersen21")
        archive.get_training_data()
        
        emulator = GPEmulator(passarchive=archive)
        return emulator

    elif emulator_label=='Cabayol23':
        print('Select emulator used in Cabayol-Garcia et al. 2023')
        emuparams = ['Delta2_p', 'n_p', 'mF', 'sigT_Mpc', 'gamma', 'kF_Mpc']
        if train==True:
            print('Train emulator')
            archive = pnd_archive.archivePND(sim_suite="Cabayol23")
            archive.get_training_data()

            emulator = NNEmulator(archive,emuparams, nepochs=nepochs_nn, step_size=75, kmax_Mpc=4, ndeg=5, Nsim=30, train=True)
        if train==False:
            if model_path==None:
                raise ValueError('if train==False, a model path must be provided')
            else:
                print('Load pre-trained emulator')
                archive = pnd_archive.archivePND(sim_suite="Cabayol23")
                archive.get_training_data()
                emulator = NNEmulator(archive,emuparams, train=False,model_path=model_path)
                
    if emulator_label==None:
        print('Select custom emulator')
        if (archive==None)|(emu_algorithm==None):
            raise ValueError('If an emulator label is not provided, the archive and emulator algorithm are required')

        if (emu_algorithm=='NN')&(archive_label=='Gadget'):
            emuparams = ['Delta2_p', 'n_p', 'mF', 'sigT_Mpc', 'gamma', 'kF_Mpc']
            emulator = NNEmulator(archive,emuparams,ndeg=ndeg, zmax=zmax, kmax_Mpc=kmax_Mpc,save_path=save_path )

        if (emu_algorithm=='NN')&(archive_label=='Nyx'):
            emuparams = ['Delta2_p', 'n_p','mF', 'sigT_Mpc', 'gamma', 'lambda_P']
            raise Exception('Work in progress')

        if (emu_algorithm=='GP')&(archive_label=='Gadget'):
            emuparams = ['Delta2_p', 'n_p', 'mF', 'sigT_Mpc', 'gamma', 'kF_Mpc']
            emulator = GPEmulator(passarchive=archive)

                
    return emulator

## LOAD EMULATOR IN PEDERSEN+21

In [20]:
emulator = P1D_emulator(emulator_label='Pedersen21')

Select emulator used in Pedersen et al. 2021
Training GP on 330 points
GPs optimised in 0.49 seconds


## LOAD EMULATOR IN CABAYOL+23

### train the emulator

In [24]:
emulator = P1D_emulator(emulator_label='Cabayol23',nepochs_nn=1)

Select emulator used in Cabayol-Garcia et al. 2023
Train emulator
start the training of the emulator
Training network on 9900
Emualtor trained in 1.3434910774230957 seconds


### Load trained emulator

In [21]:
emulator = P1D_emulator(emulator_label='Cabayol23', train=False, model_path='NNmodels/NNEmulator_LaCEHC.pt')

Select emulator used in Cabayol-Garcia et al. 2023
Load pre-trained emulator
Model loaded. No training needed


## LOAD CUSTOM EMULATOR

In [25]:
emulator = P1D_emulator()

Select custom emulator


ValueError: If an emulator label is not provided, the archive and emulator algorithm are required

In [26]:
emulator = P1D_emulator(archive=archive, emu_algorithm='NN')

Select custom emulator
start the training of the emulator
Training network on 330
Emualtor trained in 5.417848825454712 seconds


In [27]:
emulator = P1D_emulator(archive=archive, emu_algorithm='GP')

Select custom emulator
Training GP on 330 points
GPs optimised in 0.50 seconds
